# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


1. What test is appropriate for this problem? Does CLT apply?

The appropriate test for this problem is a bootstrap hypothesis test of two samples. The test statistic will be the mean of the rate of call backs for both white-sounding and black-sounding names. Yes, the Central Limit Theorem applies. 

2. What are the null and alternative hypotheses?

The null hypothesis is that race has no impact on the rate of call backs. 
The alternative hypothesis is that race has an impact on the rate of call backs.

In [18]:
# Callback rates for both races
b_nocall = len(data[data.race=='b'])
w_nocall = len(data[data.race=='w'])
b_call = sum(data[data.race=='b'].call)
w_call = sum(data[data.race=='w'].call)
b_rate = b_call / b_nocall
w_rate = w_call / w_nocall
print('Callback rates for blacks & whites are: {:1.3f}, {:1.3f}'.format(b_rate, w_rate))

Callback rates for blacks & whites are: 0.064, 0.097


3. Compute margin of error, confidence interval, and p-value.

In [17]:
w = data[data.race=='w']
b = data[data.race=='b']

def bootstrap_replicate_1d(data, func):
    return func(np.random.choice(data, size=len(data)))

def draw_bs_reps(data, func, size=1):
    """Draw bootstrap replicates."""

    # Initialize array of replicates: bs_replicates
    bs_replicates = np.empty(size)

    # Generate replicates
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1d(data, func)

    return bs_replicates

# Take 10,000 bootstrap replicates of the mean: bs_replicates
bs_replicates = draw_bs_reps(w.call, np.mean, 10000)

# Compute and print Margin of Error
m = np.std(w.call) / np.sqrt(len(w.call))
print('Margin of error for white call backs', m)

# Compute the 95% confidence interval: conf_int
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Print the confidence interval
print('95% confidence interval for white call backs =', conf_int)

# Take 10,000 bootstrap replicates of the mean: bs_replicates
bs_replicates = draw_bs_reps(b.call, np.mean, 10000)

# Compute and print Margin of Error
m = np.std(b.call) / np.sqrt(len(b.call))
print('Margin of error for black call backs', m)

# Compute the 95% confidence interval: conf_int
conf_int = np.percentile(bs_replicates, [2.5, 97.5])

# Print the confidence interval
print('95% confidence interval for black call backs =', conf_int)

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

# Compute difference of mean call backs: empirical_diff_means
empirical_diff_means = diff_of_means(w.call, b.call)

# Compute mean of all forces: mean_force
mean_force = np.mean(data.call)

# Generate shifted arrays
call_w_shifted = w.call - np.mean(w.call) + mean_force
call_b_shifted = b.call - np.mean(b.call) + mean_force

# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates_a = draw_bs_reps(call_w_shifted, np.mean, size=10000)
bs_replicates_b = draw_bs_reps(call_b_shifted, np.mean, size=10000)

# Get replicates of difference of means: bs_replicates
bs_replicates = bs_replicates_a - bs_replicates_b

# Compute and print p-value: p
p = np.sum(bs_replicates >= empirical_diff_means) / len(bs_replicates)
print('p-value =', p)

Margin of error for white call backs 0.0059840016981803105
95% confidence interval for white call backs = [0.08459959 0.10841889]
Margin of error for black call backs 0.004977108869798699
95% confidence interval for black call backs = [0.0550308  0.07433265]
p-value = 0.0


4. Write a story describing the statistical significance in the context or the original problem.

Resumes with black sounding names have an average call back rate of 6.4 percent and resumes with white sounding names have a call back rate of 9.7 percent. The margin of errors calculated after running the the bootstrap sample test do not overlap. This, coupled with such a low p-value, shows there is a significant difference in call back rates for black and white sounding names. We can reject the null hypothesis and it is safe to assume racism still has a significant presence in the labor market.

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

This analysis by no means says racism is the most important factor in call back success. Now, it is certainly possible racism is the most important factor in call back success, but similar test on other variables must be conducted to confirm. At first glance, I believe years of experience, education and number of jobs will have a greater impact, and exploring those variables in greater depth is where I would start. 